https://github.com/spatialaudio/paper-aes154-individual-hrtf-hoa2binaural

# HRTF Individualised Mag-LS and COMPASS Ambisonics-To-Binaural Rendering: Overall Perceived Quality for Pre-Conditioned Listeners

Frank Schultz, Shaimaa Doma, Sascha Spors, Janina Fels

precis reviewed, open access paper for 154th Audio Eng. Soc. Conv., Espoo, 2023, Paper xxxxx, http://www.aes.org/e-lib/browse.cfm?elib=xxxxx

- plots of raw data
- statistical evaluation

license model:
- code under MIT License
- project under https://creativecommons.org/licenses/by/4.0/

In [ ]:
# python version 3.11.0   
import numpy as np  # version 1.24.1
import json
import matplotlib.pyplot as plt  #  version 3.6.2 
from matplotlib import rcParams
import os
from matplotlib.patches import Rectangle
import pandas as pd  # version 1.5.3
from hypothesize.compare_groups_with_single_factor import rmmcppb  # version 1.1.0 or 1.1.1
from hypothesize.utilities import trim_mean

In [ ]:
# taken from Nara Hahn's AES148 shelving filter code
# https://github.com/spatialaudio/aes148-shelving-filter
# cf.
# https://matplotlib.org/stable/tutorials/text/usetex.html
def set_rcparams():
    """Plot helping."""
    rcParams['axes.linewidth'] = 0.5
    rcParams['axes.edgecolor'] = 'black'
    rcParams['axes.facecolor'] = 'None'
    rcParams['axes.labelcolor'] = 'black'
    #
    rcParams['xtick.color'] = 'black'
    rcParams['xtick.major.size'] = 0
    rcParams['xtick.major.width'] = 1
    rcParams['xtick.minor.size'] = 0
    rcParams['xtick.minor.width'] = 1
    #
    rcParams['ytick.color'] = 'black'
    rcParams['ytick.major.size'] = 0
    rcParams['ytick.major.width'] = 1
    rcParams['ytick.minor.size'] = 0
    rcParams['ytick.minor.width'] = 1
    #
    rcParams['grid.color'] = 'silver'
    rcParams['grid.linestyle'] = '-'
    rcParams['grid.linewidth'] = 0.33
    #
    rcParams['legend.frameon'] = 'False'
    #
    rcParams['font.family'] = 'serif'
    rcParams['font.size'] = 8
    rcParams['text.usetex'] = True
    rcParams['text.latex.preamble'].join([r'\usepackage{amsmath}',
                                         r'\usepackage{gensymb}'])
    rcParams['legend.title_fontsize'] = 8
    #
    rcParams['savefig.bbox'] = 'tight'
    rcParams['savefig.pad_inches'] = 0.05
    rcParams['savefig.facecolor'] = 'white'
    rcParams['savefig.dpi'] = 300

In [ ]:
results_folder = "ratings_raw_data/"

scene_config = ['1 Speech Sparta MagLS 7th',
                '2 Music Sparta MagLS 7th',
                '3 Speech Compass 3rd',
                '4 Music Compass 3rd',
                '5 Speech Sparta MagLS 3rd',
                '6 Music Sparta MagLS 3rd',
                '7 Speech Compass 1st',
                '8 Music Compass 1st',
                '9 Speech Sparta MagLS 1st',
                '10 Music Sparta MagLS 1st',
                '11 Speech Compass 2nd',
                '12 Music Compass 2nd',
                '13 Speech Sparta MagLS 2nd',
                '14 Music Sparta MagLS 2nd']

configs = ['Speech Sparta', 'Speech Compass', 'Music Sparta', 'Music Compass']

hrtf_config = ['own 3/7', 'own', 'kemar hats', 'aachen hats', 'random ind 1', 'random ind 2']
hrtf_config_short = ['own', 'own', 'kem', 'ac', 'r1', 'r2']

participant_id = ['ID_09_Test1', 'ID_09_Test2', 'ID_09_Test3',
                  'ID_09_Test4', 'ID_09_Test5', 'ID_09_Test6']

scenes = len(scene_config)
hrtfs = len(hrtf_config)
participants = len(participant_id)

#col = ['silver', 'linen', 'lightgrey', 'seashell', 'gainsboro', 'snow']
col = ['ghostwhite', 'powderblue', 'aliceblue', 'lightblue', 'silver', 'linen']
set_rcparams()

In [ ]:
r = np.zeros((scenes, hrtfs, participants))
print(r.shape)
for cnt, val in enumerate(participant_id):
    with open(results_folder+"binaural_test_"+val+".json", 'r') as file:
        data = json.load(file)
        # print(val)
        # print(data)
        for s in range(scenes):
            r[s, :, cnt] = data["scene_"+str(s+1)]["ratings"]

In [ ]:
speech = r[0::2, :, :]
print(speech.shape)
speech_config = scene_config[::2]
speech_config

In [ ]:
music = r[1::2, :, :]
print(music.shape)
music_config = scene_config[1::2]
music_config

In [ ]:
# we need to sort HOA orders as they appear not sorted in scene_config
sparta_idx = [0, 2, 6, 4]
compass_idx = [1, 5, 3]

In [ ]:
speech_sparta = speech[sparta_idx, :, :]
print(speech_sparta.shape)
speech_sparta_config = [speech_config[idx] for idx in sparta_idx]
speech_sparta_config

In [ ]:
music_sparta = music[sparta_idx, :, :]
print(music_sparta.shape)
music_sparta_config = [music_config[idx] for idx in sparta_idx]
music_sparta_config

In [ ]:
speech_compass = speech[compass_idx, :, :]
print(speech_compass.shape)
speech_compass_config = [speech_config[idx] for idx in compass_idx]
speech_compass_config

In [ ]:
music_compass = music[compass_idx, :, :]
print(music_compass.shape)
music_compass_config = [music_config[idx] for idx in compass_idx]
music_compass_config

In [ ]:
def plot_hrtf_patches():
    # comment for reshape order='F' ToDo
    data_r = np.reshape(
        data, (data.shape[0]*data.shape[1], data.shape[2]), order='F')
    print(data_r.shape)

    fig, ax = plt.subplots(1, 1)
    fig.set_size_inches(12, 3)
    
    n_boot = 10000
    ax.boxplot(data_r.T,
               showmeans=True,
               notch=True,
               bootstrap=n_boot,
               labels=data_r.shape[0] * [''])
    # rectangle patches to separate HRTFs:
    for p in range(hrtfs):
        ax.add_patch(Rectangle((0.5+len(config)*p, 0),
                               len(config), 100,
                               fc=col[p],
                               ec=col[p],
                               lw=1))
        # label HRTFs:
        ax.text(1+len(config)*p, 102,
                hrtf_config[p], horizontalalignment='left')
    # label for HOA order:
    hoa = []
    for i in range(len(config)):
        hoa.append(config[i][-3:])
    hoa_r = hoa * 6
    for i in range(len(hoa)):
        hoa_r[i] = hoa[0]
    for cnt, val in enumerate(hoa_r):
        ax.text(cnt+1, -7, val, horizontalalignment='center')

    ax.set_xlim(0.5, data_r.shape[0]+0.5)
    ax.set_ylim(-10, 110)
    ax.set_ylabel('Overall Perceived Quality')
    for tmp in configs:
        if config[0].find(tmp) > -1:
            title_str = tmp
            break
    ax.set_title(title_str)

In [ ]:
if True:
    data = speech_sparta
    config = speech_sparta_config
    plot_hrtf_patches()

    data = music_sparta
    config = music_sparta_config
    plot_hrtf_patches()

    data = speech_compass
    config = speech_compass_config
    plot_hrtf_patches()

    data = music_compass
    config = music_compass_config
    plot_hrtf_patches()

In [ ]:
def plot_scene_patches():
    # comment for reshape order='C' ToDo
    data_r = np.reshape(
        data, (data.shape[0]*data.shape[1], data.shape[2]), order='C')
    print(data_r.shape)
    fig, ax = plt.subplots(1, 1)
    # AES: 16cm width, height such that all 4 figures fit to one page
    fig.set_size_inches(16/2.54, 16/2.54 * 0.275)  
    
    # rectangle patches to separate scenes:
    for p in range(len(config)):
        ax.add_patch(Rectangle((0.5+hrtfs*p, 0), hrtfs, 100, fc=col[p], ec=col[p], lw=1))
    
    # manual grid lines above patch but below boxplot:
    for k in np.arange(10,100, 10):
        ax.plot([1, data_r.shape[0]],[k, k], ':', color='darkgrey', lw=0.25)
    
    #https://matplotlib.org/stable/gallery/statistics/boxplot.html
    medianprops = dict(linestyle='-', linewidth=1.5, color='navy')
    meanlineprops = dict(linestyle='dashed', linewidth=1, color='C1')
    flierprops = dict(marker='x', markersize=3,
                      markerfacecolor='black', 
                      markeredgecolor='black')
    ax.boxplot(data_r.T,
               showmeans=True,
               meanline=True,
               meanprops=meanlineprops,
               medianprops=medianprops,
               flierprops=flierprops,
               labels=data_r.shape[0] * [''])
    
    for cnt, val in enumerate(hrtf_config_short * data.shape[0]):
        ax.text(cnt+1, -7, val, horizontalalignment='center')

    for tmp in configs:
        if config[0].find(tmp) > -1:  # find string in string
            patch_str = tmp
            break  # once found we are done
    for cnt, val in enumerate(config):
        ax.text(cnt*hrtfs+1, 102, patch_str + ' HOA ' +
                val[-3:], horizontalalignment='left')
        # indicate highest order at my own ref
        ax.text(cnt*hrtfs+1, 0.5, config[0][-3:], horizontalalignment='center')
       
    ax.set_xlim(0.5, data_r.shape[0]+0.5)
    ax.set_ylim(-10, 110)
    ax.set_ylabel('Overall Perceived Quality')
    plt.tight_layout()

In [ ]:
folder = 'graphics/'

data = speech_sparta
config = speech_sparta_config
plot_scene_patches()
plt.savefig(folder+'speech_sparta_aes154.pdf', dpi=600, bbox_inches='tight')

data = music_sparta
config = music_sparta_config
plot_scene_patches()
plt.savefig(folder+'music_sparta_aes154.pdf', dpi=600, bbox_inches='tight')

data = speech_compass
config = speech_compass_config
plot_scene_patches()
plt.savefig(folder+'speech_compass_aes154.pdf', dpi=600, bbox_inches='tight')

data = music_compass
config = music_compass_config
plot_scene_patches()
plt.savefig(folder+'music_compass_aes154.pdf', dpi=600, bbox_inches='tight')

In [ ]:
for i in range(r.shape[0]):
#for i in range(1):
    #i = 1
    print('###########\n', scene_config[i])
    df = pd.DataFrame(r[i, :, :].T, columns = hrtf_config)
    # print(df)
    # 0.2 -> 20 % trimmed mean parameter, nboot is chosen based on number of contrasts
    res = rmmcppb(df, trim_mean, 0.2,
                  alpha=.05,
                  con=None,
                  dif=True,
                  nboot=None,
                  BA=False,
                  hoch=False,
                  SR=False,
                  seed=False)
    print('significant results:', res['num_sig'])
    print(res['output'])
    print(res['output']['p_value'] < res['output']['p_crit'])
    print('significant, i.e. reject H0(groups are equal)')
    print(res['con'][:,res['output']['p_value'] < res['output']['p_crit']])
    print('non-significant, i.e. it is very likely that groups do not differ')
    print(res['con'][:,res['output']['p_value'] >= res['output']['p_crit']])